# **<span style='color:#F1A424'>Innavation Workshop - 2023. Quick success</span>**
#### **<span style='color:#F1A424'>Practical part of Quick Success - TelegramBot, Skoltech, Moscow-2023</span>**

**Instructors:** Anna Petrovskaia, Mikhail Gasanov and Elizveta Kiseleva

--------

### 1. Register a Telegram bot

Register your bot in Telegram and get a token.

1. To register a new bot, launch the BotFather bot and send the command:

`/newbot`

2. In the name field, specify the name of the bot being created, for example, `Crop Bot`. This name will be seen by users when communicating with the bot.

3. In the username field, specify the username of the bot being created, for example `Crop_bot`. By user name, you will be able to find the bot in Telegram. The user name must end in `...Bot` or `..._bot`.

4. As a result, you will receive a token. Save it, it will be needed in the future.

5. Install the icon for the bot — the `logo.png` file. Send the BotFather bot the command:

`/setuserpic`

6. Save token to file.

In [1]:
import json

with open(".token_telegram.json", "w") as file:
    creds = {"token": "<YOUR UNIQUE TOKEN FROM BOT FATHER>"}
    json.dump(creds, file)

## 2. Download pretrained model from Drive


Example of file URL

`https://drive.google.com/file/d/1Fg0ZbTeQryYfCB-m-iTUMGdm0B6Y9_Io/view?usp=drive_link`

In [ ]:
# use unique if from gdrive - example: 1Fg0ZbTeQryYfCB-m-iTUMGdm0B6Y9_Io
!gdown "file_id"

## 3. Install packages for telegram bot


__aiogram__

aiogram is a modern and fully asynchronous framework for Telegram Bot API written in Python 3.8 using asyncio and aiohttp.

https://aiogram.dev/


__nest-asyncio__

By design asyncio does not allow its event loop to be nested. This presents a practical problem: When in an environment where the event loop is already running it’s impossible to run tasks and wait for the result. Trying to do so will give the error “RuntimeError: This event loop is already running”.

https://github.com/erdewit/nest_asyncio

In [ ]:
!pip3 -q install nest_asyncio
!pip3 -q install aiogram

     |████████████████████████████████| 196 kB 15.6 MB/s 
     |████████████████████████████████| 8.8 MB 65.5 MB/s 


## 4. Add your Network

Class for your neural net

In [ ]:
class Network(nn.Module):

    # Network Initialisation
    def __init__(self, params):

        super(Network, self).__init__()

        Cin, Hin, Win = params["shape_in"]
        init_f = params["initial_filters"]
        num_fc1 = params["num_fc1"]
        num_classes = params["num_classes"]
        self.dropout_rate = params["dropout_rate"]

        # Convolution Layers
        self.conv1 = nn.Conv2d(Cin, init_f, kernel_size=3)
        h, w = findConv2dOutShape(Hin, Win, self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2 * init_f, kernel_size=3)
        h, w = findConv2dOutShape(h, w, self.conv2)
        self.conv3 = nn.Conv2d(2 * init_f, 4 * init_f, kernel_size=3)
        h, w = findConv2dOutShape(h, w, self.conv3)
        self.conv4 = nn.Conv2d(4 * init_f, 8 * init_f, kernel_size=3)
        h, w = findConv2dOutShape(h, w, self.conv4)

        # compute the flatten size
        self.num_flatten = h * w * 8 * init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def forward(self, X):

        # Convolution & Pool Layers
        X = Functional.relu(self.conv1(X))
        X = Functional.max_pool2d(X, 2, 2)
        X = Functional.relu(self.conv2(X))
        X = Functional.max_pool2d(X, 2, 2)
        X = Functional.relu(self.conv3(X))
        X = Functional.max_pool2d(X, 2, 2)
        X = Functional.relu(self.conv4(X))
        X = Functional.max_pool2d(X, 2, 2)

        X = X.view(-1, self.num_flatten)

        X = Functional.relu(self.fc1(X))
        X = Functional.dropout(X, self.dropout_rate)
        X = self.fc2(X)
        return Functional.log_softmax(X, dim=1)


def findConv2dOutShape(hin, win, conv, pool=2):
    # get conv arguments
    kernel_size = conv.kernel_size
    stride = conv.stride
    padding = conv.padding
    dilation = conv.dilation

    hout = np.floor(
        (hin + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) / stride[0] + 1
    )
    wout = np.floor(
        (win + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) / stride[1] + 1
    )

    if pool:
        hout /= pool
        wout /= pool
    return int(hout), int(wout)

## 5. Edit bot.py file and fill missed parts of code

* Open file `GreenHouseBot/bot.py`
* Edit `run_model()` function
* Save file 
* Run bot

In [2]:
import logging
import asyncio
from aiogram import Bot, Dispatcher, types
from aiogram.filters import CommandStart, Command
from aiogram.enums import ParseMode
from aiogram import F
import uuid
import sys
import time, json, io
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as Functional
import torchvision.transforms as transforms


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
credintails = json.load(open(".token_telegram.json", "r"))

TOKEN = credintails["token"]
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

# Initialize bot and dispatcher
dp = Dispatcher()

bot = Bot(TOKEN, parse_mode=ParseMode.HTML)


def run_model(image: np.array) -> int:

    ### BEGIN

    ## END YOUR SOLUTION
    device = torch.device(str("cuda:0") if torch.cuda.is_available() else "cpu")
    cnn_model = cnn_model.to(device)
    cnn_model.eval()
    my_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Resize((46, 46))]
    )
    image_tensor = my_transforms(image).unsqueeze(0)
    image_tensor = image_tensor.to(device)
    ### BEGIN YOUR SOLUTION

    ### END YOUR SOLUTION
    pred = output.argmax(dim=1, keepdim=True)
    pred = pred.cpu().detach().numpy()
    del image_tensor, output, cnn_model
    torch.cuda.empty_cache()  # Clear cache
    return pred[0][0]


@dp.message(CommandStart())
async def send_welcome(message: types.Message):
    """
    This handler will be called when user sends /start or /help command
    """
    await message.reply("Hi!\n\nI'm CropBot!\n\nI need photos 🥦.")


@dp.message(F.content_type.in_({"photo", "document"}))
async def echo(message: types.Message):
    file_in_io = io.BytesIO()
    if message.content_type == "photo":
        await bot.download(message.photo[-1], destination=file_in_io)
    elif message.content_type == "document":
        await bot.download(message.document, destination=file_in_io)
    plant_image = np.array(Image.open(io.BytesIO(file_in_io.read())))
    await bot.send_message(message.from_user.id, "Neural nets started!")
    task = run_model(image=plant_image)
    human_dict = {0: "Mint", 1: "Rausmarine"}
    await message.answer(human_dict[task])


# Handle /cancel command
@dp.message(Command("cancel"))
async def cancel_handler(message: types.Message):
    await message.reply(
        "Canceled, enter /start", reply_markup=types.ReplyKeyboardRemove()
    )


async def main() -> None:
    # And the run events dispatching
    await dp.start_polling(bot)


if __name__ == "__main__":
    asyncio.run(main())

## End ☘️
-----

## Nest asyncio

In [ ]:
import nest_asyncio

nest_asyncio.apply()